# Loan Chargeoff Prediction

This notebook provides R scripts for modelling and scoring of Loan Chargeoff Prediction solution. 

### Prerequisites
**The following steps should be executed outside of the notebook using SQL scripts that can be checked out from github. The scripts are present in SQLR folder.** The source code for a complete solution template is here: https://github.com/Microsoft/r-server-loan-chargeoff
1. *Step 1* - This step does database setup to create required databases and tables for following steps.
2. *Step 2* - This step imports the raw csv data into SQL Server and creates views/tables with feature and label columns.

### Modelling/Scoring
3. *Step 2a* (optional) - This step demonstrates feature selection and categorical transforms from MicrosoftML.
4. *Step 3* - This step is for training various MicrosoftML models on the data and calculating stats for evaluation.
5. *Step 4* - This step does batch prediction, first picking the best model and then predicting and storing the prediction results in a table.

## NOTE
* Replace appropriate username and password parameters in the connection strings

In [1]:
# Load required libraries
library(RevoScaleR)
library(MicrosoftML)

In [2]:
# Replace username and password in the connection string with appropriate values.
sql_connection_string <- "Driver=SQL Server;Server=.;Database=LoanChargeOff;UID=sqluser;PWD=Password1#"

## Step 2a (Optional): Feature Selection
This step demonstrates feature seleection and categorical transforms that can be used with MicrosoftML training algorithms

In [3]:
###########################################################################################################################################
# Function for demonstrating MicrosoftML's selectFeatures and categorical transforms.
# 
# Parameters:
#                * connection_string - substitute appropriate username and password along with database name and server if needed
#                * train_set - table name of training set (usually the prefix of 10k/100k/1m will only change based on data set size)
#                * test_set - table name of testing set
#
# Pre-Requisites:
#                Make sure Loan_ChargeOff.ps1 has been run for your appropriate size data set so the required tables have already been 
#                created and dataset imported (it's already been run for 10k loans data set)
#                   
###########################################################################################################################################
select_features <- function(connection_string = sql_connection_string,
                            train_set = "loan_chargeoff_train_10k",
                            test_set = "loan_chargeoff_test_10k")
{
    cc <- RxInSqlServer(connectionString = connection_string)
    rxSetComputeContext(cc)
    testing_set <- RxSqlServerData(table=test_set, connectionString = connection_string)
    training_set <- RxSqlServerData(table=train_set, connectionString = connection_string)
    
    features <- rxGetVarNames(testing_set)
    variables_to_remove <- c("memberId", "loanId", "payment_date", "loan_open_date", "charge_off")
    feature_names <- features[!(features %in% variables_to_remove)]
    model_formula <- as.formula(paste(paste("charge_off~"), paste(feature_names, collapse = "+")))
    selected_count <- 0
    
    ml_trans <- list(categorical(vars = c("purpose", "residentialState", "branch", "homeOwnership", "yearsEmployment")),
                    selectFeatures(model_formula, mode = mutualInformation(numFeaturesToKeep = 100)))
    candidate_model <- rxLogisticRegression(model_formula, data = training_set, mlTransforms = ml_trans)
    predicted_score <- rxPredict(candidate_model, testing_set, extraVarsToWrite = c("charge_off"))
    # set compute context to local otherwise need to store prediction in RxSqlServerData data source for RxInSqlServer compute context
    rxSetComputeContext("local")
    predicted_roc <- rxRoc("charge_off", grep("Probability", names(predicted_score), value = T), predicted_score)
    auc <- rxAuc(predicted_roc)
    
    features_to_remove <- c("(Bias)")
    selected_features <- rxGetVarInfo(summary(candidate_model)$summary)
    selected_feature_names <- names(selected_features)
    selected_feature_names[!(selected_feature_names %in% features_to_remove)]
}

In [4]:
features <- select_features()
features

Elapsed time: 00:00:00.6056614


[1] "installment"             "loanAmount"             
 [3] "past_due_5"              "term"                   
 [5] "past_due_4"              "past_due_3"             
 [7] "past_due_2"              "past_due_1"             
 [9] "past_due"                "branch.\"Bridgeport\""  
[11] "numTotalCreditLines"     "branch.\"San Antonio\"" 
[13] "annualIncome"            "residentialState.\"NV\""
[15] "branch.\"Shreveport\""   "residentialState.\"WV\""
[17] "branch.\"Spokane\""      "branch.\"Gulfport\""    
[19] "residentialState.\"VT\""

## Step 3: Model Training and Evaluation

This step does model training and computes evaluation stats which are stored in a models table

In [5]:
# Function to compute model evaluation stats
model_eval_stats <- function(scored_data, label="charge_off", predicted_prob="Probability", predicted_label="PredictedLabel")
{
  roc <- rxRoc(label, grep(predicted_prob, names(scored_data), value=T), scored_data)
  auc <- rxAuc(roc)
  crosstab_formula <- as.formula(paste("~as.factor(", label, "):as.factor(", predicted_label, ")"))
  cross_tab <- rxCrossTabs(crosstab_formula, scored_data)
  conf_matrix <- cross_tab$counts[[1]]
  tn <- conf_matrix[1,1]
  fp <- conf_matrix[1,2]
  fn <- conf_matrix[2,1]
  tp <- conf_matrix[2,2]
  accuracy <- (tp + tn) / (tp + fn + fp + tn)
  precision <- tp/(tp+fp)
  recall <- tp / (tp+fn)
  f1score <- 2 * (precision * recall) / (precision + recall)
  return(list(auc=auc, accuracy=accuracy, precision = precision, recall=recall, f1score=f1score))
}

In [6]:
###########################################################################################################################################
# Function for training of models using MicrosoftML algorithms. Feature selection is done during training using selectFeatures mlTransforms
# as well as categorical transform.
# 
# Parameters:
#                * model_name - name of the model to train_set
#                * train_set - table name of training set (usually the prefix of 10k/100k/1m will only change based on data set size)
#                * test_set - table name of testing set
#                * score_set - table name to be used for scoring the test_set table for evaluation
#                * connection_string - substitute appropriate username and password along with database name and server if needed
#
# Pre-Requisites:
#                Make sure Loan_ChargeOff.ps1 has been run for your appropriate size data set so the required tables have already been 
#                created and dataset imported (it's already been run for 10k loans data set)
#                   
###########################################################################################################################################
train_model <- function(model_name = "logistic_regression",
                        train_set = "loan_chargeoff_train_10k",
                        test_set = "loan_chargeoff_test_10k",
                        score_set = "loan_chargeoff_eval_score_10k",
                        connection_string = sql_connection_string
                        )
{

    cc <- RxInSqlServer(connectionString = connection_string)
    rxSetComputeContext(cc)
    training_set <- RxSqlServerData(table=train_set, connectionString = connection_string)
    testing_set <- RxSqlServerData(table=test_set, connectionString = connection_string)
    scoring_set <- RxSqlServerData(table=score_set, connectionString = connection_string, overwrite=TRUE)
    ##########################################################################################################################################
    ## Training and evaluating model based on model selection
    ##########################################################################################################################################
    features <- rxGetVarNames(training_set)
    vars_to_remove <- c("memberId", "loanId", "payment_date", "loan_open_date", "charge_off")
    feature_names <- features[!(features %in% vars_to_remove)]
    model_formula <- as.formula(paste(paste("charge_off~"), paste(feature_names, collapse = "+")))
    ml_trans <- list(categorical(vars = c("purpose", "residentialState", "branch", "homeOwnership", "yearsEmployment")),
                    selectFeatures(model_formula, mode = mutualInformation(numFeaturesToKeep = 100)))
    
    print(paste("Starting to train with", model_name))
    if (model_name == "logistic_reg") {
        model <- rxLogisticRegression(formula = model_formula,
                        data = training_set,
                        mlTransforms = ml_trans)
    } else if (model_name == "fast_trees") {
        model <- rxFastTrees(formula = model_formula,
                        data = training_set,
                        mlTransforms = ml_trans)
    } else if (model_name == "fast_forest") {
        model <- rxFastForest(formula = model_formula,
                        data = training_set,
                        mlTransforms = ml_trans)
    } else if (model_name == "fast_linear") {
        model <- rxFastLinear(formula = model_formula,
                        data = training_set,
                        mlTransforms = ml_trans)
    } else if (model_name == "neural_net") {
        model <- rxNeuralNet(formula = model_formula,
                        data = training_set,
                        numIterations = 42,
                        optimizer = adaDeltaSgd(),
                        mlTransforms = ml_trans)
    }
    print("Done training.")
    
    # selected features
    features_to_remove <- c("(Bias)")
    selected_features <- rxGetVarInfo(summary(model)$summary)
    selected_feature_names <- names(selected_features)
    selected_feature_filtered <- selected_feature_names[!(selected_feature_names %in% features_to_remove)]
    
    # evaluate model
    rxPredict(model, testing_set, outData = scoring_set, extraVarsToWrite = c("loanId", "payment_date", "charge_off"), overwrite=TRUE)
    print("Done writing predictions for evaluation of model.")
    list(model_name = model_name, model = model, stats = model_eval_stats(scoring_set))
}

In [7]:
# train on MicrosoftML algorithms
# Warning message about "implicit list embedding of S4 objects is deprecated" is expected
ml_algs <- c("logistic_reg", "fast_trees", "fast_forest", "fast_linear", "neural_net")
model_stats <- lapply(ml_algs, train_model)

# find the best model based on f1score
best_model <- model_stats[[which.max(sapply(model_stats, function(stat) stat$stats$f1score))]]
# save to file for use during scoring
save(best_model, file="loan_chargeoff_best_model_10k.rdata")
best_model

[1] "Starting to train with logistic_reg"
Elapsed time: 00:00:00.0261225
[1] "Done training."


Warning message in `[<-`(`*tmp*`, "outputDataSource", value = <S4 object of class structure("RxSqlServerData", package = "RevoScaleR")>):
"implicit list embedding of S4 objects is deprecated"

[1] "Done writing predictions for evaluation of model."


Warning message in `[<-`(`*tmp*`, "outputDataSource", value = <S4 object of class structure("RxSqlServerData", package = "RevoScaleR")>):
"implicit list embedding of S4 objects is deprecated"

Rows Read: 101, Total Rows Processed: 101, Total Chunk Time: 0.007 seconds 
[1] "Starting to train with fast_trees"
[1] "Done training."
Elapsed time: 00:00:00.0465263


...

Warning message in `[<-`(`*tmp*`, "outputDataSource", value = <S4 object of class structure("RxSqlServerData", package = "RevoScaleR")>):
"implicit list embedding of S4 objects is deprecated"

[1] "Done writing predictions for evaluation of model."


Warning message in `[<-`(`*tmp*`, "outputDataSource", value = <S4 object of class structure("RxSqlServerData", package = "RevoScaleR")>):
"implicit list embedding of S4 objects is deprecated"

Rows Read: 101, Total Rows Processed: 101, Total Chunk Time: 0.006 seconds 


$model_name
[1] "fast_trees"

$model
Call:
rxFastTrees(formula = model_formula, data = training_set, mlTransforms = ml_trans)

FastTreeBinaryClassification (BinaryClassifierTrainer) for: charge_off~payment+past_due+remain_balance+loanAmount+interestRate+grade+term+installment+isJointApplication+purpose+residentialState+branch+annualIncome+yearsEmployment+homeOwnership+incomeVerified+creditScore+dtiRatio+revolvingBalance+revolvingUtilizationRate+numDelinquency2Years+numDerogatoryRec+numInquiries6Mon+lengthCreditHistory+numOpenCreditLines+numTotalCreditLines+numChargeoff1year+payment_1+payment_2+payment_3+payment_4+payment_5+past_due_1+past_due_2+past_due_3+past_due_4+past_due_5+remain_balance_1+remain_balance_2+remain_balance_3+remain_balance_4+remain_balance_5
Data: training_set (RxSqlServerData Data Source) 

$stats
$stats$auc
[1] 0.9928972

$stats$accuracy
[1] 0.9936091

$stats$precision
[1] 0.8670886

$stats$recall
[1] 0.7919075

$stats$f1score
[1] 0.8277946



## Step 4: Batch prediction of Loan chargeoff
This step does batch prediction of a data set based on the best model arrived at in the previous step

In [8]:
###########################################################################################################################################
# Function for demonstrating MicrosoftML's selectFeatures and categorical transforms.
# 
# Parameters:
#                * connection_string - substitute appropriate username and password along with database name and server if needed
#                * best_models_file - file where best_model object from training/testing step is stored
#                * score_set - table name of for scoring data (usually the prefix of 10k/100k/1m will only change based on data set size)
#                * score_prediction - table name where to store prediction results
#
# Pre-Requisites:
#                1. Make sure Loan_ChargeOff.ps1 has been run for your appropriate size data set so the required tables have already been 
#                   created and dataset imported (it's already been run for 10k loans data set)
#                2. Modelling must have been completed
#                   
###########################################################################################################################################
batch_score <- function (connection_string = sql_connection_string,
                         best_models_file = "loan_chargeoff_best_model_10k.rdata",
                         score_set = "loan_chargeoff_score_10k",
                         score_prediction = "loan_chargeoff_prediction_10k")
{
    load(best_models_file)
    if (!exists("best_model"))
    {
      stop("best_models_file does not contain best_model object, make sure you saved it properly during training step.")
    }
    cc <- RxInSqlServer(connectionString = connection_string)
    rxSetComputeContext(cc)
    scoring_data <- RxSqlServerData(table = score_set, connectionString = connection_string)
    prediction_data <- RxSqlServerData(table = score_prediction, connectionString = connection_string)
    
    # Warning: this will drop and recreate the prediction table
    rxPredict(best_model$model, scoring_data, outData = prediction_data, extraVarsToWrite = c("loanId", "payment_date"), overwrite=TRUE)
}

In [9]:
# Warning message "implicit list embedding of S4 objects is deprecated" is expected.
batch_score()

Warning message in `[<-`(`*tmp*`, "outputDataSource", value = <S4 object of class structure("RxSqlServerData", package = "RevoScaleR")>):
"implicit list embedding of S4 objects is deprecated"